In [12]:
from decouple import config
import requests
import time
import base64
from openai import OpenAI

# Configuration
OPENAI_API_KEY = config("OPENAI_API_KEY")

ESP32_IP = "192.168.1.104"
CAPTURE_URL = f"http://{ESP32_IP}/capture"
COMMAND_URL = f"http://{ESP32_IP}/cmd"
INTERVAL = 0.05  # 50ms between captures

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Expected labels
valid_labels = {'up', 'down', 'left', 'right', 'stop', 'no signal'}

# The prompt for image analysis
prompt = """Analyze this image and determine the direction of the signal shown in it. 
The possible answers are: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
If you are unsure or the image does not contain a signal, return only 'no signal'. 
Provide only the exact answer as a single word: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
Do not include any explanations or additional text."""

def capture_image_from_esp32():
    """Captures image from ESP32-CAM and converts to base64"""
    try:
        response = requests.get(CAPTURE_URL, timeout=1)
        if response.status_code == 200:
            image_base64 = base64.b64encode(response.content).decode('utf-8')
            return f"data:image/jpeg;base64,{image_base64}"
    except Exception as e:
        print(f"Capture error: {e}")
        return None

def send_command_to_esp32(command):
    """Sends command to ESP32"""
    try:
        response = requests.get(f"{COMMAND_URL}?cmd={command}", timeout=1)
        if response.status_code == 200:
            print(f"Command sent: {command}")
    except Exception as e:
        print(f"Command error: {e}")

def get_direction(image_url):
    """Gets direction using OpenAI's vision model"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url,
                            },
                        },
                    ],
                }
            ],
            max_tokens=10
        )
        raw_output = response.choices[0].message.content.strip().lower()
        cleaned_output = raw_output.replace('"', '').replace("'", '')
        return cleaned_output if cleaned_output in valid_labels else "no signal"
    except Exception as e:
        print(f"AI error: {e}")
        return "no signal"

def main():
    print("Starting rapid response control system...")
    print(f"ESP32 IP: {ESP32_IP}")
    print(f"Capture interval: {INTERVAL} seconds")
    
    last_command = "stop"
    last_process_time = time.time()
    
    while True:
        current_time = time.time()
        
        if current_time - last_process_time >= INTERVAL:
            image_url = capture_image_from_esp32()
            
            if image_url:
                direction = get_direction(image_url)
                print(f"Detected direction: {direction}")
                
                if direction != last_command:
                    send_command_to_esp32(direction)
                    last_command = direction
            
            last_process_time = current_time

if __name__ == "__main__":
    main()

Starting rapid response control system...
ESP32 IP: 192.168.1.104
Capture interval: 0.05 seconds
Detected direction: no signal
Command sent: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: left
Command sent: left
Detected direction: left
Detected direction: no signal
Command sent: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: up
Command error: HTTPConnectionPool(host='192.168.1.104', port=80): Read timed out. (read timeout=1)
Detected direction: no signal
Command sent: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: down
Command sent: down
Detected direction: down
Detected direction: no signal
Command sent: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detected direction: no signal
Detecte

KeyboardInterrupt: 